## Dependencies and Spark context setup

In [1]:
import $ivy.`org.apache.spark::spark-core:3.5.0`
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import $ivy.`com.koloboke:koloboke-impl-jdk8:1.0.0`
import $ivy.`com.typesafe.akka:akka-remote_2.13:2.5.23`
import $cp.`/app/fractal-core-SPARK-3.5.0.jar`

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $cp.$

In [6]:
import org.apache.spark.sql.NotebookSparkSession
val sc = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.driver.memory", "2g")
    .config("spark.jars.packages", "com.koloboke:koloboke-impl-jdk8:1.0.0,com.typesafe.akka:akka-remote_2.13:2.5.23")
    .getOrCreate()
    .sparkContext
}
sc.setLogLevel("ERROR")

Getting spark JARs

Creating SparkSession

Spark UI

import org.apache.spark.sql.NotebookSparkSession
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@3bedbab

## Motif listing
1. Create FractalContext from SparkContext
2. Create FractalGraph: unlabeled and based on the built-in adjacency lists format
3. Create a vertex fractoid, apply 4 extension operations yielding induced subgraphs with 4 vertices
4. Aggregate subgraphs by mapping each to its non-canonical pattern (quick) and the number 1
5. Internally aggregationCanonicalPatternLong counts how many subgraphs of each canonical pattern exists in the graph

In [10]:
import br.ufmg.cs.systems.fractal.{FractalContext, FractalGraph}
val fc = new FractalContext(sc)
val fg = fc.unlabeledGraphFromAdjLists("/app/data/citeseer")
val patternCountRDD = fg 
    .vfractoid // induced subgraphs
    .extend(4) // four extension operations: induced subgraphs with 4 vertices
    .aggregationCanonicalPatternLong(s => s.quickPattern(), 0L, s => 1L, _ + _) // count by canonical pattern
patternCountRDD.collect() // collect result from resulting Spark RDD

import br.ufmg.cs.systems.fractal.{FractalContext, FractalGraph}
fc: FractalContext = br.ufmg.cs.systems.fractal.FractalContext@2ffe2b81
fg: FractalGraph = FractalGraph(
  path = "/app/data/citeseer",
  graphClass = "br.ufmg.cs.systems.fractal.graph.UnlabeledMainGraph",
  fc = br.ufmg.cs.systems.fractal.FractalContext@2ffe2b81,
  confs = Map(),
  logLevel = "warn",
  edgePredicate = null,
  vertexPredicate = null
)
patternCountRDD: org.apache.spark.rdd.RDD[(br.ufmg.cs.systems.fractal.pattern.Pattern, Long)] = ShuffledRDD[23] at foldByKey at Fractoid.scala:401
res10_4: Array[(br.ufmg.cs.systems.fractal.pattern.Pattern, Long)] = Array(
  (
    bliss{edges=[(0,1),(1,2),(0,2),(2,3),(1,3),(0,3)],vlabels=[1,1,1,1],elabels=[0,0,0,0,0,0]},
    255L
  ),
  (
    bliss{edges=[(1,2),(2,3),(1,3),(0,3)],vlabels=[1,1,1,1],elabels=[0,0,0,0]},
    22900L
  ),
  (
    bliss{edges=[(0,1),(0,2),(2,3),(1,3)],vlabels=[1,1,1,1],elabels=[0,0,0,0]},
    3094L
  ),
  (
    bliss{edges=[(1,2),(0,2),(2,3),(1,3),